## Objective: 

The goal is to create movie recommendation system.The engine was built by recommending movies based on cosine similarity using Nearest Neighbor (unsupervised) with the help of Collborative Filtering technique such as top rated content.

In [1]:
import pandas as pd
import numpy as np

In [2]:
movies_df = pd.read_csv('./movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
rating_df = pd.read_csv('./ratings.csv')
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
merged_df = pd.merge(movies_df,rating_df,on='movieId')
merged_df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [5]:
merged_df=merged_df.drop(columns=['genres','timestamp'],axis=1)
merged_df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [6]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   movieId  100836 non-null  int64  
 1   title    100836 non-null  object 
 2   userId   100836 non-null  int64  
 3   rating   100836 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 3.8+ MB


## Data Preprocessing

In [7]:
#dropping null values if present any
combine_movie_rating = merged_df.dropna(axis=0,subset=['title'])

In [8]:
# sorting how many persons rated each movie in the dataset

movie_ratingcount = combine_movie_rating.groupby(by=['title'])['rating'].count().reset_index()

In [9]:
movie_ratingcount = movie_ratingcount.rename(columns={'rating':'total_rating_count'})

In [10]:
movie_ratingcount.head()

,title,total_rating_count
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [11]:
totalrated_df = combine_movie_rating.merge(movie_ratingcount,left_on='title',right_on='title', how='left')

In [13]:
totalrated_df.head()

,movieId,title,userId,rating,total_rating_count
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [14]:
# fixing popularity threshold as 50, so consider movies rated more than 50 votes

popularity_threshold = 50
rating_popular_movie= totalrated_df.query('total_rating_count >= @ popularity_threshold')
rating_popular_movie.head()

,movieId,title,userId,rating,total_rating_count
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [15]:
# creating a pivot table

movie_features_df = rating_popular_movie.pivot_table(values='rating',index='title',columns='userId').fillna(0)

In [17]:
movie_features_df.head(10)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0
"40-Year-Old Virgin, The (2005)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5
A.I. Artificial Intelligence (2001),0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.5,0.0,4.5,0.0,3.5
"Abyss, The (1989)",4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0
Ace Ventura: Pet Detective (1994),0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,2.0,0.0,0.0,0.0,3.5,0.0,3.0


In [19]:
# now converting the pivot table into csr matrix

from scipy.sparse import csr_matrix

movie_feature_df_matrix = csr_matrix(movie_features_df.values)

## Model building

In [24]:
from sklearn.neighbors import NearestNeighbors

In [25]:
model_knn = NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(movie_feature_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

## Model Prediction

In [26]:
query_index = np.random.choice(movie_features_df.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 10)

57


In [27]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Black Hawk Down (2001):

1: Training Day (2001), with distance of 0.4670387924368733:
2: Bourne Identity, The (2002), with distance of 0.5148134520286639:
3: Shaun of the Dead (2004), with distance of 0.5194059706510228:
4: Scarface (1983), with distance of 0.5223093260793435:
5: Sin City (2005), with distance of 0.5240255790228407:
6: Bourne Supremacy, The (2004), with distance of 0.527281989684235:
7: Last Samurai, The (2003), with distance of 0.5324547013049609:
8: Star Wars: Episode II - Attack of the Clones (2002), with distance of 0.5328849257141104:
9: Departed, The (2006), with distance of 0.5368240698069544:
